In [2]:
import time
from urllib import urlencode  # TODO(Miles): Python 3
import selenium.webdriver
from bs4 import BeautifulSoup
import itertools
import pandas as pd

In [478]:
def search_boardgamegeek(query, browser, delay=10):
    search_url = "https://boardgamegeek.com/browse/boardgame/page/{}".format(query)
    browser.get(search_url)
    #time.sleep(delay)  # Wait a few seconds before getting the HTML source
    return browser.page_source

In [479]:
def get_game_url(product_tag):
    title_elements = product_tag.select('a')
    if title_elements:
        product_title = title_elements[1].get('href')
        return product_title

In [480]:
def get_boardgamegeek_product_details(query, browser):
    html = search_boardgamegeek(query, browser)
    soup = BeautifulSoup(html, 'html.parser')
    product_tags = soup.find_all("tr", id="row_")
    product_tags.extend(soup.find_all("tr", id="row_"))
    return [get_game_url(t) for t in product_tags]

In [481]:
browser = selenium.webdriver.Firefox()
product_details = []
for i in xrange(1, 136):
    product_details.append(get_boardgamegeek_product_details(i, browser)[:100])
browser.quit()

In [519]:
product_details

[u'/boardgame/161936/pandemic-legacy-season-1',
 u'/boardgame/182028/through-ages-new-story-civilization',
 u'/boardgame/12333/twilight-struggle',
 u'/boardgame/120677/terra-mystica',
 u'/boardgame/187645/star-wars-rebellion',
 u'/boardgame/169786/scythe',
 u'/boardgame/173346/7-wonders-duel',
 u'/boardgame/102794/caverna-cave-farmers',
 u'/boardgame/84876/castles-burgundy',
 u'/boardgame/3076/puerto-rico',
 u'/boardgame/31260/agricola',
 u'/boardgame/96848/mage-knight-board-game',
 u'/boardgame/25613/through-ages-story-civilization',
 u'/boardgame/115746/war-ring-second-edition',
 u'/boardgame/170216/blood-rage',
 u'/boardgame/164153/star-wars-imperial-assault',
 u'/boardgame/167791/terraforming-mars',
 u'/boardgame/2651/power-grid',
 u'/boardgame/72125/eclipse',
 u'/boardgame/121921/robinson-crusoe-adventures-cursed-island',
 u'/boardgame/124742/android-netrunner',
 u'/boardgame/178900/codenames',
 u'/boardgame/35677/le-havre',
 u'/boardgame/205059/mansions-madness-second-edition',
 

In [482]:
product_details = list(itertools.chain.from_iterable(product_details))

In [520]:
len(product_details)

13500

In [484]:
def search_boardgamegeek(query, browser, delay=10):
    search_url = "https://boardgamegeek.com{}".format(query)
    browser.get(search_url)
    #time.sleep(delay)  # Wait a few seconds before getting the HTML source
    return browser.page_source

In [582]:
def get_boardgamegeek_game_details(query, browser):
    html = search_boardgamegeek(query, browser)
    soup = BeautifulSoup(html, 'html.parser')
    product_tags = soup.find_all("div", class_="content")
    return [(get_game_title(t), get_secondary_game_info(t), get_the_rest(t)) for t in product_tags]

In [607]:
def get_game_title(product_tag):
    title_elements = product_tag.find_all("div", class_="game-header-title-info")
    if title_elements:
        game_title = title_elements[1].find_all('h1')
        year_published = game_title[0].text.split()[-1:]
        game_title = ' '.join(game_title[0].text.split()[:-1])
        return game_title, year_published

In [488]:
def get_secondary_game_info(product_tag):
    title_elements = product_tag.find_all('li', attrs={'class':'gameplay-item'})
    if title_elements:
        game_info = title_elements
        num_players = game_info[0].text.split()
        min_players = num_players[0][0]
        max_players = num_players[0][2:]
        best_players = num_players[-1]
        play_time = game_info[1].text.split()[0].split(u'\u2013')
        age = game_info[2].text.split()[1].strip('+')
        complexity = game_info[3].text.split()[2]
        return min_players, max_players, best_players, play_time, age, complexity

In [531]:
def get_the_rest(product_tag):
    title_elements = product_tag.find_all('li', attrs={'class':'outline-item'})
    if title_elements:
        game_title = title_elements
        designers = game_title[3].find_all('a')[1:]
        for i, designer in enumerate(designers):
            designers[i] = designer.text
        artists = game_title[4].find_all('a')[1:]
        for i, artist in enumerate(artists):
            artists[i] = artist.text
        publishers = game_title[5].find_all('a')[1:]
        for i, publisher in enumerate(publishers):
            publishers[i] = publisher.text
        categories = game_title[6].find_all('a')[1:]
        for i, categorie in enumerate(categories):
            categories[i] = categorie.text
        mechanisms = game_title[7].find_all('a')[1:]
        for i, mechanism in enumerate(mechanisms):
            mechanisms[i] = mechanism.text
        return designers, artists, publishers, categories, mechanisms

In [721]:
product_details[9145]

u'/boardgame/91313/ataxx'

In [759]:
product_details[13107]

u'///'

In [760]:
'''browser = selenium.webdriver.Firefox()
game_details = []
for i in product_details[1000:1001]:
    game_title = get_boardgamegeek_game_details(i, browser)[1]
    new_search = '{}/credits'.format(i)
    game_title2 = get_boardgamegeek_game_details2(new_search, browser)[1]
    game_info = tuple(game_title) + tuple(game_title2)
    all_info = [game_info[0][0], game_info[0][1][0], game_info[1][0], \
                    game_info[1][1], game_info[1][2], game_info[1][3], game_info[1][4], game_info[1][5], \
                   game_info[2], game_info[3], game_info[4], game_info[5], game_info[6]]
    if all_info[3] == '':
        all_info[3] = all_info[2]
    try:
        all_info[5][1]
    except IndexError:
        all_info[5].append(all_info[5][0])
    game_details.append(all_info)
browser.quit()'''

"browser = selenium.webdriver.Firefox()\ngame_details = []\nfor i in product_details[1000:1001]:\n    game_title = get_boardgamegeek_game_details(i, browser)[1]\n    new_search = '{}/credits'.format(i)\n    game_title2 = get_boardgamegeek_game_details2(new_search, browser)[1]\n    game_info = tuple(game_title) + tuple(game_title2)\n    all_info = [game_info[0][0], game_info[0][1][0], game_info[1][0],                     game_info[1][1], game_info[1][2], game_info[1][3], game_info[1][4], game_info[1][5],                    game_info[2], game_info[3], game_info[4], game_info[5], game_info[6]]\n    if all_info[3] == '':\n        all_info[3] = all_info[2]\n    try:\n        all_info[5][1]\n    except IndexError:\n        all_info[5].append(all_info[5][0])\n    game_details.append(all_info)\nbrowser.quit()"

In [761]:
browser = selenium.webdriver.Firefox()
game_details = []
for i in product_details[13107:13108]:
    new_search = '{}/credits'.format(i)
    game_info = get_boardgamegeek_game_details(new_search, browser)[0]
    all_info = [game_info[0][0], game_info[0][1][0], game_info[1][0], \
                    game_info[1][1], game_info[1][2], game_info[1][3], game_info[1][4], game_info[1][5], \
                   game_info[2][0], game_info[2][1], game_info[2][2], game_info[2][3], game_info[2][4]]
    if all_info[3] == '':
        all_info[3] = all_info[2]
    try:
        all_info[5][1]
    except IndexError:
        all_info[5].append(all_info[5][0])
    game_details.append(all_info)
browser.quit()

IndexError: list index out of range

In [730]:
secondary = list(game_details)

In [731]:
secondary

[[u'Ataxx',
  u'(1990)',
  u'P',
  u'ayers',
  u'none',
  [u'Time:', u'Time:'],
  u'\u2013\u2013',
  u'2.25',
  [u'Dave Crummack', u'Craig Galley'],
  [],
  [u'Capcom Co., Ltd.',
   u'Leland Corporation',
   u'Virgin Mastertronic',
   u'Wise Owl Software'],
  [u'Abstract Strategy'],
  [u'Area Enclosure']]]

In [763]:
yes = [[u'¥€$',
  u'(2004)',
  u'2',
  u'4',
  u'none',
  [u'30', u'30'],
  u'8',
  u'1.64',
  [u'Armstrong'],
  [u'Armstrong'],
  [u'Ravensburger Spieleverlag GmbH'],
  [u'Economic'],
  [u'Dice Rolling']]]

In [768]:
yes_df = pd.DataFrame\
(yes, index=range(13107, 13108), columns=[ \
                         'Title', 'Year Published',\
                         'Min Players', 'Max Players', 'Best Players',\
                         'Play Time', 'Age', 'Complexity', 'Designers',\
                         'Artists', 'Publishers', 'Categories',\
                         'Mechanisms'])

In [769]:
yes_df

,Title,Year Published,Min Players,Max Players,Best Players,Play Time,Age,Complexity,Designers,Artists,Publishers,Categories,Mechanisms
13107,¥€$,(2004),2,4,none,"[30, 30]",8,1.64,[Armstrong],[Armstrong],[Ravensburger Spieleverlag GmbH],[Economic],[Dice Rolling]


In [770]:
all_data = almost_all.append(yes_df).sort_index()

In [771]:
all_data

,Title,Year Published,Min Players,Max Players,Best Players,Play Time,Age,Complexity,Designers,Artists,Publishers,Categories,Mechanisms
1,Pandemic Legacy: Season 1,(2015),2,4,4,"[60, 60]",13,2.81,"[Rob Daviau, Matt Leacock]",[Chris Quilliams],"[Z-Man Games, Asterion Press, Devir, Filosofia...","[Environmental, Medical]","[Action Point Allowance System, Co-operative P..."
2,Through the Ages: A New Story of Civilization,(2015),2,4,3,"[180, 240]",14,4.29,[Vlaada Chvátil],"[Jakub Politzer, Milan Vavroň]","[Cranio Creations, Czech Games Edition, Devir,...","[Card Game, Civilization, Economic]","[Action Point Allowance System, Auction/Biddin..."
3,Twilight Struggle,(2005),2,2,2,"[180, 180]",13,3.53,"[Ananda Gupta, Jason Matthews]","[Viktor Csete, Rodger B. MacGowan, Chechu Niet...","[GMT Games, (Self-Published), Asterion Press, ...","[Modern Warfare, Political, Wargame]","[Area Control / Area Influence, Campaign / Bat..."
4,Terra Mystica,(2012),2,5,4,"[60, 150]",12,3.93,"[Jens Drögemüller, Helge Ostertag]",[Dennis Lohausen],"[Feuerland Spiele, Bard Centrum Gier, Cranio C...","[Civilization, Economic, Fantasy, Territory Bu...","[Route/Network Building, Variable Phase Order,..."
5,Star Wars: Rebellion,(2016),2,4,2,"[180, 240]",14,3.56,[Corey Konieczka],[],"[ADC Blackfire Entertainment, Asterion Press, ...","[Movies / TV / Radio theme, Science Fiction, W...","[Area Control / Area Influence, Area Movement,..."
6,Scythe,(2016),1,5,4,"[90, 115]",14,3.32,[Jamey Stegmaier],[Jakub Rozalski],"[Stonemaier Games, Albi, Feuerland Spiele, Fir...","[Civilization, Economic, Exploration, Fighting...","[Area Control / Area Influence, Grid Movement,..."
7,7 Wonders Duel,(2015),2,2,2,"[30, 30]",10,2.25,"[Antoine Bauza, Bruno Cathala]",[Miguel Coimbra],"[Repos Production, ADC Blackfire Entertainment...","[Ancient, Card Game, City Building, Civilization]","[Card Drafting, Set Collection]"
8,Caverna: The Cave Farmers,(2013),1,7,4,"[30, 210]",12,3.79,[Uwe Rosenberg],[Klemens Franz],"[Lookout Games, 999 Games, Crowd Games, Devir,...","[Animals, Economic, Fantasy, Farming]","[Tile Placement, Worker Placement]"
9,The Castles of Burgundy,(2011),2,4,2,"[30, 90]",12,3.04,[Stefan Feld],"[Julien Delval, Harald Lieske]","[alea, Ravensburger Spieleverlag GmbH, Broadwa...","[Dice, Medieval, Territory Building]","[Dice Rolling, Set Collection, Simultaneous Ac..."
10,Puerto Rico,(2002),2,5,4,"[90, 150]",12,3.29,[Andreas Seyfarth],"[Harald Lieske, Franz Vohwinkel]","[alea, Ravensburger Spieleverlag GmbH, Albi, B...","[City Building, Economic, Farming]",[Variable Phase Order]


In [745]:
almost_all = first_13107.append(final_bit)

In [748]:
almost_all = almost_all.append(ataxx)

In [756]:
almost_all.iloc[9140:9150]

,Title,Year Published,Min Players,Max Players,Best Players,Play Time,Age,Complexity,Designers,Artists,Publishers,Categories,Mechanisms
9141,Subbuteo Cricket,(1949),2,2,2,"[30, 30]",––,2.00,[Peter Adolph],[],"[Subbuteo Sports Games, LTD]","[Action / Dexterity, Collectible Components, S...",[]
9142,DEER LORD!,(2015),4,12,6,"[30, 90]",12,1.67,[],[],[(Self-Published)],"[Bluffing, Card Game, Humor, Party Game, Print...","[Acting, Storytelling, Voting]"
9143,Secret Moon,(2014),5,8,none,"[10, 10]",10,1.11,[Seiji Kanai],[Noboru Sugiura],"[Arclight, Kanai Factory, minimalGames]",[Deduction],"[Partnerships, Variable Player Powers]"
9144,Search for Gnomes,(2012),2,4,2–4,"[35, 35]",8,1.60,[Jurgen Spreutels],[Jurgen Spreutels],[Simployt Games],"[Bluffing, Math, Memory, Party Game, Print & P...",[Memory]
9145,Ataxx,(1990),P,ayers,none,"[Time:, Time:]",––,2.25,"[Dave Crummack, Craig Galley]",[],"[Capcom Co., Ltd., Leland Corporation, Virgin ...",[Abstract Strategy],[Area Enclosure]
9145,Faras,(2014),2,5,3–4,"[30, 30]",6,1.67,[Filip Miłuński],[],[Granna],[Memory],"[Hand Management, Memory]"
9147,Tet Offensive,(1991),2,4,none,"[120, 120]",––,3.10,[Frank Chadwick],[],[GDW Games],"[Vietnam War, Wargame]","[Area Movement, Dice Rolling]"
9148,The Prestel New York Architecture Game,(2002),2,5,none,"[30, 30]",8,1.13,[Thomas Fackler],[Gottfried Müller],[Prestel Verlag],"[Educational, Memory, Real-time]","[Memory, Set Collection]"
9149,HexAgony,(2014),1,3,none,"[20, 20]",8,2.00,[Andy Allen],[],[New Entertainment Ltd],[Abstract Strategy],"[Co-operative Play, Hand Management, Pattern B..."
9150,Sag's mit Symbolen,(2005),3,6,none,"[45, 45]",12,1.00,[Claude Weber],[Hans-Jörg Brehm],[AMIGO Spiel + Freizeit GmbH],"[Humor, Party Game]","[Co-operative Play, Pattern Recognition]"


In [772]:
all_data.to_csv('boardgame_data.csv', encoding='utf-8')

In [779]:
all_data['Mechanisms']

1        [Action Point Allowance System, Co-operative P...
2        [Action Point Allowance System, Auction/Biddin...
3        [Area Control / Area Influence, Campaign / Bat...
4        [Route/Network Building, Variable Phase Order,...
5        [Area Control / Area Influence, Area Movement,...
6        [Area Control / Area Influence, Grid Movement,...
7                          [Card Drafting, Set Collection]
8                       [Tile Placement, Worker Placement]
9        [Dice Rolling, Set Collection, Simultaneous Ac...
10                                  [Variable Phase Order]
11       [Card Drafting, Hand Management, Variable Phas...
12       [Card Drafting, Co-operative Play, Deck / Pool...
13       [Action Point Allowance System, Auction/Biddin...
14       [Area Control / Area Influence, Area Movement,...
15       [Action Point Allowance System, Area Control /...
16       [Dice Rolling, Grid Movement, Modular Board, P...
17       [Card Drafting, Hand Management, Tile Placemen.

ATAXX IS ALSO A VIDEO GAME, product_details[9145]

UNABLE TO COLLECT DATA WITH PRIOR METHOD

WILL COME BACK TO THIS LATER

***GOT ATAXX INFO***

ALSO, GAME CALLED ¥€$ SHOULD SEARCH https://boardgamegeek.com////credits, NOT WHATEVER IS IN product_details[13107]. NEED TO FIX THIS LATER.

***IMPORTED ¥€$ DATA MANUALLY***